***
***
# 1. Data Preparation

## 1.1 Data slicing (for easy download)

- **Idea**: cut all files into 100 copies so that they can be downloaded at the same time
- **Input**:
```python
input_files = ['/nfs/research/goldman/zihao/Datas/p1/File_5_Annot.txt',
               '/nfs/research/goldman/zihao/Datas/p1/File_5_Consensus.txt',
               '/nfs/research/goldman/zihao/Datas/p1/File_5_Coverage.txt']
```
- **Output**:
```python
output_directories = ['/nfs/research/goldman/zihao/Datas/p1/File_5_annot/Datas/batch*.txt',
                      '/nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Datas/batch*.txt',
                      '/nfs/research/goldman/zihao/Datas/p1/File_5_coverage/Datas/batch*.txt']
```
- **Location**： 
```python
/nfs/research/goldman/zihao/errorsProject_1/1_Download/Data_slicing.py
```

```python
import csv
import os

# Define the paths for the input files and output directories
input_files = ['/nfs/research/goldman/zihao/Datas/p1/File_5_Annot.txt',
               '/nfs/research/goldman/zihao/Datas/p1/File_5_Consensus.txt',
               '/nfs/research/goldman/zihao/Datas/p1/File_5_Coverage.txt']
output_directories = ['/nfs/research/goldman/zihao/Datas/p1/File_5_annot/Datas',
                      '/nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Datas',
                      '/nfs/research/goldman/zihao/Datas/p1/File_5_coverage/Datas']

# Define the batch size for each file
batch_size = 100

# Loop through the input files and split them into batches
for input_file, output_directory in zip(input_files, output_directories):
    with open(input_file, 'r') as f:
        reader = csv.reader(f, delimiter='\t')
        header = next(reader)
        rows = [row for row in reader]

    num_batches = len(rows) // batch_size + 1
    for i, batch in enumerate(range(0, len(rows), batch_size)):
        filename = f"batch_{i+1}.txt"
        batch_rows = rows[batch:batch+batch_size]
        with open(os.path.join(output_directory, filename), 'w', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(header)
            writer.writerows(batch_rows)
        del batch_rows
```

***
## 1.2 Data Download

- **Idea**: Download the corresponding sample files from the files cut in the previous steps while following the FTP link


- **Input&Output**:
```python
for use_case in "File_5_annot" "File_5_consensus" "File_5_coverage"; do
      input_folder_path="$input_base_path/$use_case/Datas"
      output_folder_path="$output_base_path/$use_case/Downloads"
```
- **Location**： 
```python
/nfs/research/goldman/zihao/errorsProject_1/1_Download/run_data_download.sh
/nfs/research/goldman/zihao/errorsProject_1/1_Download/D_file_5_script.py
```

```bash
#!/bin/bash

cd /nfs/research/goldman/zihao/errorsProject_1/1_Download

# Define the common parts of the input and output folder paths
input_base_path="/nfs/research/goldman/zihao/Datas/p1"
output_base_path="/nfs/research/goldman/zihao/Datas/p1"

# Loop over the different use cases
for use_case in "File_5_annot" "File_5_consensus" "File_5_coverage"; do
  input_folder_path="$input_base_path/$use_case/Datas"
  output_folder_path="$output_base_path/$use_case/Downloads"

  # Loop over the batches
  for i in {1..100}; do
    input_file="batch_${i}.txt"
    
    # Call the script with bsub command
    bsub -M 2000 python3 D_file_5_script.py -i "$input_folder_path/$input_file" -o "$output_folder_path"
  done
done
```

```python
import argparse
import os
import csv
import urllib.request
import gc

parser = argparse.ArgumentParser(description='Download files from FTP path')
parser.add_argument('--input-txt', '-i', required=True, help='Path to the input txt file')
parser.add_argument('--output-folder', '-o', required=True, help='Path to the output folder to save the downloaded files')

args = parser.parse_args()
input_file_path = args.input_txt
output_folder_path = args.output_folder


def download_files_from_ftp(input_file_path, output_folder_path, max_retries=5):
    with open(input_file_path, 'r') as f:
        reader = csv.DictReader(f, delimiter='\t')
        files_downloaded = False
        download_attempts = 0
        while not files_downloaded and download_attempts < 10:
            files_downloaded = True
            download_attempts += 1
            for row in reader:
                ftp_path = row['FTP_path']
                file_name = ftp_path.split('/')[-1]
                file_path = os.path.join(output_folder_path, file_name)
                if os.path.exists(file_path):
                    continue
                for i in range(max_retries):
                    try:
                        urllib.request.urlretrieve(ftp_path, file_path)
                        break
                    except:
                        if i == max_retries - 1:
                            pass
                        else:
                            continue
                if not os.path.exists(file_path):
                    files_downloaded = False
                    break
            # Force garbage collection at the end of each loop iteration
            gc.collect()


download_files_from_ftp(input_file_path, output_folder_path)
```

***
***
# 2. Data processing (adjusting sequence format)

1. Put all the downloaded consensus sequences in a single fasta file (let’s call it all_consensuses.fasta ).
    
2. Run mafft with the special options we mentioned before including --keeplength and using the reference I sent you MN908947.3 , let’s call the output all_consensuses_aligned.fasta .
    
3. Remove the reference sequence from the mafft alignment output (it should be the first sequence in the file), let’s call the resulting file all_consensuses_aligned_noReference.fasta .
    
4. Run my script createMapleFile.py on  all_consensuses_aligned_noReference.fasta WITHOUT using option --reference . This will create a MAPLE file for you, and a new reference to use with MAPLE.

***
## 2.1. Decompress

- **Idea**: Decompress the downloaded file
- **Input**: All files downloaded in the previous step

- **Output**: 
    - all_consensuses_batch_*.fasta
- **Location**： 
```python
/nfs/research/goldman/zihao/errorsProject_1/Consensuses/Decompress.py
```

```python
import os
import gzip
from Bio import SeqIO
import glob

# Define the input folder and output folder to be used
input_folder = '/nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Downloads/'
output_folder = '/nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Decompress/'

# List all gzip files in the input folder
file_list = [f for f in os.listdir(input_folder) if f.endswith('.gz')]

# Calculate the number of files to be written
num_files = 20

# Calculate the number of files in each batch
files_per_batch = (len(file_list) + num_files - 1) // num_files

# Process files in batches
for i in range(num_files):
    # Get the list of files for the current batch
    batch_files = file_list[i*files_per_batch : (i+1)*files_per_batch]
    
    # Create an output file to store the fasta sequences of all sequences in the current batch
    output_file = os.path.join(output_folder, f"all_consensuses_batch_{i+1}.fasta")
    
    with open(output_file, "w") as out_file:
        # Iterate over the files in the current batch
        for file_name in batch_files:
            
            # Get the sequence name
            seq_name = file_name.split('_')[0]

            # Check if the file exists
            if not os.path.isfile(os.path.join(input_folder, file_name)):
                print(f"File {file_name} does not exist. Skipping.")
                continue

            # Open the compressed file
            with gzip.open(os.path.join(input_folder, file_name), "rt") as f:

                # Use the SeqIO module to parse the fasta file
                sequences = SeqIO.parse(f, "fasta")

                # Write the fasta sequences to the output file
                for seq_record in sequences:
                    # Modify the sequence name to be the file name
                    seq_record.id = seq_name
                    seq_record.description = ""
                    SeqIO.write(seq_record, out_file, "fasta")
    
    print(f"Completed processing batch {i+1} of files. Output file: {os.path.basename(output_file)}")
    
    # Free up memory
    del batch_files
    del out_file
    del sequences
    del seq_record
    del f
```

### 2.1.2 Sequence Alignment

- **Idea**: Alignment using MAFFT software and according to MN908947.3 as reference

- **Input**: All files decompressed in the previous step

- **Output**: 
    - /nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Decompress/aligned_*.fasta
- **Location**： 
```python
/nfs/research/goldman/zihao/errorsProject_1/Consensuses/Aligned.sh
```

##### Code block:
```bash
cd /nfs/research/goldman/zihao/errorsProject_1/Consensuses
sh Aligned.sh
```

```bash
#!/bin/bash

for i in {1..20}
do
  mafft --6merpair --keeplength --addfragments all_consensuses_$i.fasta ref_MN908947.3.fasta > all_consensuses_aligned_$i.fasta
done

```

***
## 2.2. Del ref

- **Idea**: Delete reference MN908947.3

- **Input**: All files aligned in the previous step

- **Output**: 
    - all_consensuses_aligned_noReference_aligned_*.fasta
- **Location**： 
```python
/nfs/research/goldman/zihao/errorsProject_1/Consensuses/Del_ref.py
```

```python
import os
import gzip
from Bio import SeqIO
import glob

# Define the input folder and output folder to be used
input_folder = '/nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Decompress/Aligned/'
output_folder = '/nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Decompress/Aligned/noReference/'

# Traverse all fasta files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.fasta'):
        # Create the output file name
        output_file = os.path.join(output_folder, 'all_consensuses_aligned_noReference_' + filename[:-6] + '.fasta')
        if not os.path.exists(output_file):
            open(output_file, 'w').close()
        input_file = os.path.join(input_folder, filename)
        # Read the file content
        with open(input_file) as f:
            content = f.read()
        # Find the first occurrence of '>MN908947.3'
        start_index = content.find('>MN908947.3')
        # Find the next occurrence of '>'
        end_index = content.find('>', start_index + 1)
        # Remove the content within the specified range
        content = content[:start_index] + content[end_index:]
        # Check if there are still headers with '>MN908947.3'
        while '>MN908947.3' in content:
            start_index = content.find('>MN908947.3')
            end_index = content.find('>', start_index + 1)
            if end_index == -1:
                print('The header for the next sequence is not found in the file content')
                break
            else:
                next_header = content[end_index:].split('\n', 1)[0]
                print(f"The header for the next sequence is: {next_header}")
                content = content[:start_index] + content[end_index:]
        # Write the processed content to the output file
        with open(output_file, 'w') as f:
            f.write(content)
        # Delete the content variable
        del content
        # Print a success message
        print(f"{filename} has been processed and saved to {os.path.basename(output_file)}")
        
print('Delete is completed')
```

***
## 2.3. Merge + Remove blank

- Input: all_consensuses_aligned_noReference_aligned_*.fasta

- Output: 
    - After merge: all_consensuses_aligned_noReference.fasta

    - After Remove: rm_blank_all_consensuses_aligned_noReference.fasta
- **Location**： 
```python
/nfs/research/goldman/zihao/errorsProject_1/Consensuses/Merge_Remove_blank.py
```

```python
import os
import gzip
from Bio import SeqIO
import glob

# Define the input folder path and output file path for concatenation
concat_input_folder = '/nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Decompress/Aligned/noReference'
concat_output_file = '/nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Decompress/Aligned/noReference/all_consensuses_aligned_noReference.fasta'

# Get a list of all input file paths that start with 'all_consensuses_aligned_noReference_aligned_'
concat_input_files = glob.glob(os.path.join(concat_input_folder, 'all_consensuses_aligned_noReference_aligned_*.fasta'))

# Concatenate all input files into a single file
with open(concat_output_file, 'w') as concat_f_out:
    for concat_input_file in concat_input_files:
        with open(concat_input_file) as concat_f_in:
            concat_f_out.write(concat_f_in.read())

    # Print a success message
    print(f"All input files starting with 'all_consensuses_aligned_noReference_aligned_' have been merged and saved to {os.path.basename(concat_output_file)}")


# Define the input and output file paths for sequence filtering
filter_input_file = "/nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Decompress/Aligned/noReference/all_consensuses_aligned_noReference.fasta"
filter_output_file = "/nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Decompress/Aligned/noReference/rm_blank_all_consensuses_aligned_noReference.fasta"
removed_file = "/nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Decompress/Aligned/noReference/blank_bigger_than_20.fasta"

removed_count = 0

# Filter sequences based on the percentage of gaps and N's
with open(filter_input_file, "rU") as filter_input_handle, open(filter_output_file, "w") as filter_output_handle, open(removed_file, "w") as removed_handle:
    for record in SeqIO.parse(filter_input_handle, "fasta"):
        sequence = str(record.seq)
        n_count = sequence.count("n") + sequence.count("N")
        gap_count = sequence.count("-")
        total_count = n_count + gap_count
        if total_count/len(sequence) <= 0.2:
            SeqIO.write(record, filter_output_handle, "fasta")
        else:
            print(f"{record.id} has {total_count} gaps or N's and is being removed.")
            removed_handle.write(record.format("fasta"))
            removed_count += 1

print(f"Removed {removed_count} sequences.")
```

***
## 2.4. Transform as the MAPLE format

Run the script createMapleFile.py on  all_consensuses_aligned_noReference.fasta WITHOUT using option --reference . This will create a MAPLE file for you, and a new reference to use with MAPLE.

##### Code block:
```bash
bsub "/hps/software/users/goldman/pypy3/pypy3.7-v7.3.5-linux64/bin/pypy3 
createMapleFile.py 
--path /nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Decompress/Aligned/noReference/ 
--fasta rm_blank_all_consensuses_aligned_noReference.fasta 
--output MAPLE_format_consensuses.txt"
```

### Explanation of MAPLE format
#### > SRR20944325

|         |         |         |
|---------|---------|---------|
| - | 1 | 2 |
| n | 3 | 29901 |



- The second column number tells you the genome position the entry refers to.

- The third column
    - If the third column is not present, then the entry refers to only one position.
    - If the third column is present, its value tells you how many positions that entry refers to

- For example
|         |         |         |
|---------|---------|---------|
| n | 541 | 10 |

    - means that “n” is present in the sequence in ten consecutive positions from position 541 up to position 550.

***
***
# 3. Run MAPLE program

##### Code block:
```bash
bsub -g /MapleRealErrors -q long -M 40000 
-o /nfs/research/goldman/zihao/errorsProject_1/MAPLE/MAPLE_realData_errorChecking_output.txt 
-e /nfs/research/goldman/zihao/errorsProject_1/MAPLE/MAPLE_realData_errorChecking_error.txt 
/hps/software/users/goldman/pypy3/pypy3.7-v7.3.5-linux64/bin/pypy3 
/nfs/research/goldman/demaio/fastLK/code/MAPLEv0.3.2.py 
--model UNREST --rateVariation --estimateSiteSpecificErrorRate 
--input /nfs/research/goldman/zihao/Datas/p1/File_5_consensus/Decompress/Aligned/noReference/MAPLE_format_consensuses_new_1.txt 
--overwrite --estimateErrors --calculateLKfinalTree 
--output /nfs/research/goldman/zihao/errorsProject_1/MAPLE/MAPLE0.3.1_rateVar_errors_realData_checkingErrors
```